## <font color=#B2D732> <span style="background-color: #4424D6"> Imports

In [2]:
import sys,json
import glob, os
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/code/")

from seed_to_voxels import Seed2voxels

%matplotlib inline
%load_ext autoreload
%autoreload 2

## <font color=#B2D732> <span style="background-color: #4424D6"> Initialization

In [12]:
with open('../config/config_CL.json') as config_file:
    config = json.load(config_file) # load config file

seed_names=['brain_L-BA4'] # define seed names ex: 'spinalcord_ICA-C4'
target_name=['brain_mask'] # define structure target
seed2voxels=Seed2voxels(config,seed_names,target_name) # initial the function

## <font color=#B2D732> <span style="background-color: #4424D6"> Data extraction </font></span>
### <font color=#4424D6> I Select files </font>

In [22]:
data_seed=[];data_target=[];ts_seed_txt=[];ts_target_txt=[]
for seed_name in seed_names:
    for subject_name in config['list_subjects']:
        data_seed.append(glob.glob(config["coreg_dir"] + 'sub-'+ subject_name +'/'+seed_name.split('_')[0]+'/*' + config["coreg_tag"][seed_name.split('_')[0]] +'*.gz')[0])
        data_target.append(glob.glob(config["coreg_dir"] + 'sub-'+ subject_name +'/'+target_name[0].split('_')[0]+'/*' + config["coreg_tag"][target_name[0].split('_')[0]] +'*.gz')[0])
        ts_seed_txt.append(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+seed_name+'/timeseries/sub_' + subject_name + '_mask_' + seed_name.split('_')[-1] + '_timeseries')
        ts_target_txt.append(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+target_name[0]+'/timeseries/sub_' + subject_name + '_mask_' + target_name[0] + '_timeseries')

#mask_seed=glob.glob("/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/ICA/spinalcord_9/Comp_masked/zCanICA_23sbj_spinalcord_component_1_bin.nii.gz")[0]#glob.glob(config["main_dir"] + config["templates"][seed_name.split('_')[0]+"_seeds"] + seed_name.split('_')[-1].split('_')[-1] +'*.gz')[0]
mask_seed=glob.glob(config["main_dir"] + config["templates"][seed_name.split('_')[0]+"_seeds"] + seed_name.split('_')[-1].split('_')[-1] +'*.gz')[0]
mask_target=glob.glob(config["main_dir"] + config["masks"][target_name[0].split('_')[0]])[0]
 
# create output directory if not exist
if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+seed_name):
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+seed_name)
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+seed_name+'/timeseries/') # folder to store timeseries extraction
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+seed_name+'/brain_fc_maps/') # folder to store maps of FC

### <font color=#4424D6> II. Time series extraction </font>

In [24]:
seed_timeseries,seed_timeseries_mean,seed_timeseries_pc1=seed2voxels.extract_data(img=data_seed, mask=mask_seed, timeseries_txt=ts_seed_txt,
                                                                run="extract",n_jobs=8) # run the analyse for target voxels
target_timeseries,target_timeseries_mean,target_timeseries_pc1=seed2voxels.extract_data(img=data_target, mask=mask_target, timeseries_txt=ts_target_txt,
                                                                run="load",n_jobs=8) # run the analyse for target voxels


## <font color=#B2D732> <span style="background-color: #4424D6"> Correlation analysis

In [27]:
output_img={};
for seed_name in seed_names:
    output_img[seed_name]=[]
    output_img[seed_name]=config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/' + seed_name + '/' + target_name[0].split('_')[0]+ '_fc_maps/' + str(len(config['list_subjects'])) + 'subjects_seedmean_' + seed_name.split('_')[-1] + '_zcorr_.nii'

seed2voxels.correlation_maps(seed_timeseries_mean,target_timeseries,mask=mask_target,output_img=output_img[seed_name],Fisher=True,n_jobs=8)

## <font color=#B2D732> <span style="background-color: #4424D6"> Smoothing

In [28]:
from nilearn import image
img_smoothed=image.smooth_img(output_img[seed_name],[6,6,6])
img_smoothed.to_filename(output_img[seed_name].split('.')[0] + '_s.nii' )